In [ ]:
import kagglehub
import shutil
import os

# Download latest version
path = kagglehub.dataset_download("gpreda/covid-world-vaccination-progress")

print("Path to dataset files:", path)
## Esto lo agrege yo
destino = '/content/vacunas/'

if not os.path.exists(destino):
  shutil.copytree(path, destino)
  print('Datos copiados a: ', destino)

else:
  print('La carpeta ya existe.')

Path to dataset files: /kaggle/input/covid-world-vaccination-progress
La carpeta ya existe.


## Importaciones y Visualización Inicial

Importaciones básicas de lectura y lamado de librerias.

In [ ]:
import pandas as pd
import numpy as np



# A) Extracción del Archivo

archivo = 'country_vaccinations.csv'
ruta_archivo = os.path.join(destino, archivo)

data = pd.read_csv(ruta_archivo)

## B) Retorno de estructura del DataFrame

data.info()
#data.head() Desabilitado ya que es más útil aquí info, para ver los dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86512 entries, 0 to 86511
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              86512 non-null  object 
 1   iso_code                             86512 non-null  object 
 2   date                                 86512 non-null  object 
 3   total_vaccinations                   43607 non-null  float64
 4   people_vaccinated                    41294 non-null  float64
 5   people_fully_vaccinated              38802 non-null  float64
 6   daily_vaccinations_raw               35362 non-null  float64
 7   daily_vaccinations                   86213 non-null  float64
 8   total_vaccinations_per_hundred       43607 non-null  float64
 9   people_vaccinated_per_hundred        41294 non-null  float64
 10  people_fully_vaccinated_per_hundred  38802 non-null  float64
 11  daily_vaccinations_per_milli

## Conversiones y Aislamiento de Datos

Conversiones solicitadas en el punto B, dado que la variable 'date' era 'object'
y no tenia formato datetime64, seguro es por el size del archivo.

In [ ]:
data['date'] = pd.to_datetime(data['date']) ## B.1) Conversión a datetime

tipos_vacunas = data['vaccines'].unique() ## Extracción de marcas de vacunas
listado_paises = data['iso_code'].unique() ## Extracción de iso-codes de paises
fecha_objetivo = '2021-01-25' ## Fecha proporcionada por el ejercicio

aplicadas = { # Diccionarios para facilitar el almacenado por compañia
}

aplicadas_pais = { # Diccionario para almacenar los iso-codes por pais y cantidades

}

## Recorrido de los diccionarios y carga de datos

Por cada diccionario, se realiza un filtrado en pandas por nombre y se aplica una función de sumatoria.

El espacio de memoria debería ser mayor, pero opte por almacenar en DataFrames nuevos, de esta forma ganamos verbosidad y es amigable con el ojo.

Es importante anotar, que dado que ambos provienen de 'daily_vaccinations' se puede meter un .sum() al dataframe original para obtener el resultado mundial.


In [ ]:
for vacuna in tipos_vacunas: # Cálculo y almacenamiento de las marcas y sus vacunas aplicadas

  total = data[data['vaccines'] == vacuna]['daily_vaccinations'].sum()
  aplicadas[vacuna] = total

for iso_pais in listado_paises: ## Cálculo y almacenamiento de vacunas aplicadas por país

  total = data[data['iso_code'] == iso_pais]['daily_vaccinations'].sum()
  aplicadas_pais[iso_pais] = total

## DataFrame para marcas de Vacunas, no solicitado pero decidi almacenarlos en DFs
data_vacunas = pd.DataFrame.from_dict(aplicadas, orient = 'index', columns = ['total_aplicadas'])

## Reinicio el indice, y a la columna anterior se renombra por vacuna
data_vacunas = data_vacunas.reset_index().rename(columns = {'index' : 'vacuna'})
## DataFrame para paises, por iso code
frame_paises = pd.DataFrame.from_dict(aplicadas_pais, orient = 'index', columns = ['total_aplicadas'])
frame_paises = frame_paises.reset_index().rename(columns = {'index' : 'iso_code'})

In [ ]:
data_vacunas

,vacuna,total_aplicadas
0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",10072170.0
1,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",25046010.0
2,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",22451663.0
3,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech",620164884.0
4,Oxford/AstraZeneca,56887281.0
...,...,...
79,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",23704599.0
80,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, ...",43932105.0
81,"Abdala, Sinopharm/Beijing, Sinovac, Soberana02...",37861146.0
82,"Abdala, Moderna, Oxford/AstraZeneca, Pfizer/Bi...",201816053.0


## Calculos de totales

Utilizamos los dataframes creados para obtener la cantidad de vacunas aplicadas en todo el mundo.

Para el punto del ejercicio (e), basta con llamar el data frame, que contiene todos los paises por "ISO CODE" y sus totales acumulados. El ejercicio menciona "Promedio" pero la rubrica menciona por 10 puntos:

"10 pts.
Se obtuvo la
cantidad
correcta de
vacunas
aplicadas por
país."

In [ ]:
world_total = int(frame_paises['total_aplicadas'].sum()) ## D) Sumatoria para total de vacunas se debe hacer desde el original daily

world_total_daily = int(data['daily_vaccinations'].sum())

print(f'El World total del Daily Vacs es {world_total_daily}')

print(f'El total de vacunas aplicadas fue de {world_total} vacunas.')

frame_paises # E) Contiene los Totales por Paises, confundido ...¿Promedios, referente a qué?

El total de vacunas aplicadas fue de 11320239871 vacunas.


,iso_code,total_aplicadas
0,AFG,5741998.0
1,ALB,2748980.0
2,DZA,13697360.0
3,AND,151499.0
4,AGO,17435547.0
...,...,...
218,OWID_WLS,6859138.0
219,WLF,12538.0
220,YEM,794952.0
221,ZMB,3367782.0


## Cálculo del día Objetivo

Solo realizamos un filtrado, por la 'fecha objetivo', viriable que declare bien al inicio.

fecha_objetivo = '2021-01-25'

In [ ]:
dia_objetivo = int(data[(data['date'] == fecha_objetivo)]['daily_vaccinations'].sum()) ## F) Total de vacunas aplicadas el día seleccionado

print(f'Durante el día {fecha_objetivo} se aplicarón {dia_objetivo} vacunas.')

Durante el día 2021-01-25 se aplicarón 4470780 vacunas.


## Creación de Data Frames Alternos

A continuación se crean todos los dataframes alternos, utilizando el metodo copy(), dado que si, igualaramos estariamos creando una referencia a memoria. Por lo que copiar el dataframe original es la solución correcta.

### Cálculo de conDiferencias

Es crucial mencionar que debemos mantener la operación de pandas nativa, pero dejar claro en la documentación que no estamos manejando un cero, si no hay cambios o diferencias, dado que es invalido asumir un cero si hay NaN.

Para este caso opte por dejar los valores vacios y acotarlo en el texto.

In [ ]:
## G) Creación de nuevo dataframe y añadido de columna
conDiferencias = data.copy() ## Crucial usar copy para no referenciar.

# La columna 'conDiferencias' representa la diferencia entre el valor limpio y el valor original.
# No se imputan valores nulos; si alguno de los dos valores es NaN, la diferencia también será NaN.
# Esta decisión preserva la integridad del dato original y permite identificar posibles omisiones o errores de captura.

conDiferencias['conDiferencias'] = conDiferencias['daily_vaccinations'] - conDiferencias['daily_vaccinations_raw']

## H) Periodos de tiempo

minimo_date = data['date'].min()
maximo_date = data['date'].max()

periodo_temporal = maximo_date - minimo_date

print(f'El periodo temporal entre el periodo mínimo ({minimo_date}) '
      f'y el periodo temporal de mayor rango ({maximo_date}) es de: \n'
      f'{periodo_temporal}'
      )

El periodo temporal entre el periodo mínimo (2020-12-02 00:00:00) y el periodo temporal de mayor rango (2022-03-29 00:00:00) es de: 
482 days 00:00:00


## DataFrame conDiferencias

Este dataframe, contiene la diferencia entre las vacunas reportadas aplicadas por cada fila, dado que, existian ajustes aplicados durante los periodos de capturas, ya que podría en un punto en especifico un país no reportar datos y al día de reportar, estos tenian inconsistencias y se suavizaban los datos.

In [ ]:
conDiferencias

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,conDiferencias
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,NaN
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,NaN
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,NaN
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86507,Zimbabwe,ZWE,2022-03-25,8691642.0,4814582.0,3473523.0,139213.0,69579.0,57.59,31.90,23.02,4610.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...,-69634.0
86508,Zimbabwe,ZWE,2022-03-26,8791728.0,4886242.0,3487962.0,100086.0,83429.0,58.25,32.38,23.11,5528.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...,-16657.0
86509,Zimbabwe,ZWE,2022-03-27,8845039.0,4918147.0,3493763.0,53311.0,90629.0,58.61,32.59,23.15,6005.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...,37318.0
86510,Zimbabwe,ZWE,2022-03-28,8934360.0,4975433.0,3501493.0,89321.0,100614.0,59.20,32.97,23.20,6667.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...,11293.0


## DataFrame conCantidad

Este data frame, sirve para aplicar una función lambda que nos permita obtener el formato correspondiente con una coma en cada fila.

In [ ]:
## I) Data Frame con Cantidad

## Revisar con split.

conCantidad = data.copy()

##
conCantidad['canVac'] = conCantidad['daily_vaccinations'].apply(
    lambda x: f'{int(x):,}' if pd.notnull(x) else None
)

conCantidad

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,canVac
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,None
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,"1,367"
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,"1,367"
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,"1,367"
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,"1,367"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86507,Zimbabwe,ZWE,2022-03-25,8691642.0,4814582.0,3473523.0,139213.0,69579.0,57.59,31.90,23.02,4610.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...,"69,579"
86508,Zimbabwe,ZWE,2022-03-26,8791728.0,4886242.0,3487962.0,100086.0,83429.0,58.25,32.38,23.11,5528.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...,"83,429"
86509,Zimbabwe,ZWE,2022-03-27,8845039.0,4918147.0,3493763.0,53311.0,90629.0,58.61,32.59,23.15,6005.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...,"90,629"
86510,Zimbabwe,ZWE,2022-03-28,8934360.0,4975433.0,3501493.0,89321.0,100614.0,59.20,32.97,23.20,6667.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...,"100,614"



# DataFrame antes20

Este DF nos permite filtrar por fecha aprovechando date con su formato datetime64.



In [ ]:
## J) Data Frames antes del 20

fecha_objetivo_antes = '2020-12-20'

## antes20 = data[data['date'] < fecha_objetivo_antes], incorrecto hace referencia

antes20 = data.copy()

antes20 = antes20[antes20['date'] < fecha_objetivo_antes]

antes20

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
13403,Canada,CAN,2020-12-14,5.0,5.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html
13404,Canada,CAN,2020-12-15,727.0,727.0,NaN,722.0,722.0,0.00,0.00,NaN,19.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html
13405,Canada,CAN,2020-12-16,3025.0,3025.0,NaN,2298.0,1510.0,0.01,0.01,NaN,40.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html
13406,Canada,CAN,2020-12-17,7279.0,7279.0,NaN,4254.0,2425.0,0.02,0.02,NaN,64.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html
13407,Canada,CAN,2020-12-18,11296.0,11296.0,NaN,4017.0,2823.0,0.03,0.03,NaN,74.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82362,United States,USA,2020-12-15,84638.0,76984.0,6281.0,49771.0,27175.0,0.03,0.02,0.0,82.0,"Johnson&Johnson, Moderna, Pfizer/BioNTech",Centers for Disease Control and Prevention,https://data.cdc.gov/Vaccinations/COVID-19-Vac...
82363,United States,USA,2020-12-16,244549.0,231496.0,6757.0,159911.0,71420.0,0.07,0.07,0.0,215.0,"Johnson&Johnson, Moderna, Pfizer/BioNTech",Centers for Disease Control and Prevention,https://data.cdc.gov/Vaccinations/COVID-19-Vac...
82364,United States,USA,2020-12-17,517161.0,496980.0,7507.0,272612.0,121718.0,0.16,0.15,0.0,367.0,"Johnson&Johnson, Moderna, Pfizer/BioNTech",Centers for Disease Control and Prevention,https://data.cdc.gov/Vaccinations/COVID-19-Vac...
82365,United States,USA,2020-12-18,933551.0,904656.0,8718.0,416390.0,180653.0,0.28,0.27,0.0,544.0,"Johnson&Johnson, Moderna, Pfizer/BioNTech",Centers for Disease Control and Prevention,https://data.cdc.gov/Vaccinations/COVID-19-Vac...


## K) Data Frame Pfizer

In [ ]:
pfizer = data.copy()

pfizer['vaccines'] = pfizer['vaccines'].str.lower() ## Para normalizar

conteo = pfizer['vaccines'].str.contains('pfizer', na = False).sum()

## Solo académico, no solicitado
condicion_negativa = (~pfizer['vaccines'].str.contains('pfizer', na = False)).sum()
print('----- Bloque extra -----')
print(f'El valor de la condicion negativa es: {condicion_negativa}')
print('-----Fin del Bloque extra----')
## Fin del Bloque extra

ratio = (conteo/len(pfizer))*100
print(f'El conteo de las vacunas Pfizer es: {conteo} de {len(pfizer)} registros \n'
      f'Es decir, representa {ratio:.2f}% de las vacunas aplicadas.'
      )
##Lo hago así para comprender mejor el termino de series, que usa indices
condicion = pfizer['vaccines'].str.contains('pfizer', na = False) ## Serie Booleana

pfizer = pfizer[condicion]

pfizer

----- Bloque extra -----
El valor de la condicion negativa es: 22319
-----Fin del Bloque extra----
El conteo de las vacunas Pfizer es: 64193 de 86512 registros 
Es decir, representa 74.20% de las vacunas aplicadas.


,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,"johnson&johnson, oxford/astrazeneca, pfizer/bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"johnson&johnson, oxford/astrazeneca, pfizer/bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"johnson&johnson, oxford/astrazeneca, pfizer/bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"johnson&johnson, oxford/astrazeneca, pfizer/bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"johnson&johnson, oxford/astrazeneca, pfizer/bi...",World Health Organization,https://covid19.who.int/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85069,Wales,OWID_WLS,2022-03-24,6921195.0,2526832.0,2397677.0,1756.0,1705.0,218.33,79.71,75.64,538.0,"moderna, oxford/astrazeneca, pfizer/biontech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/vaccin...
85070,Wales,OWID_WLS,2022-03-25,6923298.0,2527100.0,2398097.0,2103.0,1667.0,218.40,79.72,75.65,526.0,"moderna, oxford/astrazeneca, pfizer/biontech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/vaccin...
85071,Wales,OWID_WLS,2022-03-26,6923706.0,2527184.0,2398322.0,408.0,1519.0,218.41,79.72,75.66,479.0,"moderna, oxford/astrazeneca, pfizer/biontech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/vaccin...
85072,Wales,OWID_WLS,2022-03-27,6925183.0,2528051.0,2398676.0,1477.0,1505.0,218.46,79.75,75.67,475.0,"moderna, oxford/astrazeneca, pfizer/biontech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/vaccin...


## Almacenamiento de los DataFrames en Excel

In [ ]:
print('Proceso de almacenamiento inicializado')

with pd.ExcelWriter('resultadoReto.xlsx') as writer:
  conDiferencias.to_excel(writer, sheet_name = 'conDiferencias')
  conCantidad.to_excel(writer, sheet_name = 'conCantidad')
  antes20.to_excel(writer, sheet_name = 'antes20')
  pfizer.to_excel(writer, sheet_name = 'Pfizer')

print('Proceso Completado de almacenamiento')

Exception ignored in: <function ZipFile.__del__ at 0x7bb58f7f8c20>
Traceback (most recent call last):
  File "/usr/lib/python3.11/zipfile.py", line 1895, in __del__
    self.close()
  File "/usr/lib/python3.11/zipfile.py", line 1912, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


# Dato Curioso

Un compañero buscaba montar su drive para dejar de depender de exportar los CSVs esto se logra con el código de a continuación, solo hay que tener cuidado con dar permisos excesivos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.flush_and_unmount()